# Исследование сервиса для чтения книг по подписке

Цель: анализ базы данных сервиса для чтения книг по подписке.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student',
 'pwd': 'Sdf4$2;d-d30pp',
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432,
 'db': 'data-analyst-final-project-db'}

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db']) 

In [4]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [5]:
def select (sql):
    return pd.read_sql(sql, con=engine)

In [6]:
query = ''' SELECT 
            * 
            FROM books 
            LIMIT 5
         '''
select(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [7]:
query = ''' SELECT 
            * 
            FROM authors
            LIMIT 5
         '''
select(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [8]:
query = ''' SELECT 
            * 
            FROM ratings
            LIMIT 5
         '''
select(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [9]:
query = ''' SELECT 
            * 
            FROM reviews 
            LIMIT 5
         '''
select(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [10]:
query = ''' SELECT 
            * 
            FROM publishers 
            LIMIT 5
         '''
select(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


- сколько книг вышло после 1 января 2000 года;

In [11]:
query = ''' SELECT 
            COUNT(DISTINCT book_id) as cnt
            FROM books
            WHERE
            publication_date > '2000-01-01'
         '''
select(query)

,cnt
0,819


Количество обзоров и среднюю оценку для каждой книги;

In [12]:
query = ''' SELECT
            SUBQ1.book_id,
            SUBQ1.average_score,
            SUBQ.amt_reviews
            FROM
           (SELECT 
            book_id,
            AVG(DISTINCT rating) as average_score           
            FROM ratings
            GROUP BY
            book_id) AS SUBQ1
            LEFT JOIN 
            (SELECT
            book_id,
            COUNT(DISTINCT review_id) as amt_reviews
            FROM reviews
            GROUP BY
            book_id) AS SUBQ
            ON SUBQ1.book_id = SUBQ.book_id
            
         '''
select(query)

,book_id,average_score,amt_reviews
0,1,3.666667,2.0
1,2,2.500000,1.0
2,3,4.500000,3.0
3,4,4.500000,2.0
4,5,4.000000,4.0
...,...,...,...
995,996,4.000000,3.0
996,997,3.666667,3.0
997,998,3.333333,4.0
998,999,4.500000,2.0


Издательство, которое выпустило наибольшее число книг толще 50 страниц 

In [13]:
query = ''' SELECT
            publishers.publisher,
            SUBQ.cnt_books
            FROM publishers
            INNER JOIN
            (SELECT 
            publisher_id,
            COUNT(book_id) as cnt_books
            FROM books
            WHERE num_pages > 50
            GROUP BY
            publisher_id
            ORDER BY cnt_books DESC
            LIMIT 1) AS SUBQ
            ON publishers.publisher_id = SUBQ.publisher_id
         '''
select(query)

,publisher,cnt_books
0,Penguin Books,42


Автор с самой высокой средней оценкой книг — для книг с 50 и более оценками;

In [17]:
query = ''' SELECT
            authors.author_id,
            authors.author,
            ROUND(top_author.avg, 2) AS average_rating
            FROM authors
            RIGHT JOIN
            (SELECT
            author_id,
            AVG(rating_avg)
            FROM books
            LEFT JOIN
            (SELECT 
            AVG (rating) AS rating_avg,
            book_id
            FROM ratings
            GROUP BY
            book_id
            ) AS ratings ON books.book_id = ratings.book_id
            WHERE books.book_id IN (SELECT 
            books.book_id
            FROM books
            LEFT JOIN ratings ON books.book_id = ratings.book_id
            GROUP BY books.book_id
            HAVING COUNT(rating_id) >= 50)
            GROUP BY author_id
            ORDER BY avg DESC
            LIMIT 1
            ) AS top_author ON authors.author_id = top_author.author_id       
         '''
select(query)

,author_id,author,average_rating
0,236,J.K. Rowling/Mary GrandPré,4.28


Среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [16]:
query = '''  SELECT
            AVG (cnt_reviews) as avg_reviews
            FROM
            (SELECT
            username
            FROM ratings
            GROUP BY
            username
            HAVING COUNT(DISTINCT rating_id) > 50) AS SUBQ1   
            LEFT JOIN
            (SELECT 
            username,
            COUNT (DISTINCT review_id) as cnt_reviews
            FROM reviews
            GROUP BY username) AS SUBQ
            ON SUBQ1.username = SUBQ.username
            
         '''
select(query)

,avg_reviews
0,24.333333


# Вывод

В данном исследовании были проанализированны данные и получены следующие результаты:
- после 1 января 2000 года вышло 819 книг;
- издательство, которое выпустило наибольшее число книг толще 50 страниц: Penguin Books
- J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг;
- 24.3 среднее количество обзоров от пользователей, которые поставили больше 50 оценок.